# Preparing data
Since the [FFBP package](https://github.com/alex-ten/pdpyflow/tree/master/FFBP) runs [Tensorflow](https://www.tensorflow.org/), input data must be made readable for the underlying computational graph. FFBP has a special class called `InputData` that lets you easily convert a csv data file into a structure that can be injected into a Tensorflow graph.

The csv file must be structured in a certain way (see example below). The very first row is reserved for column labels; it is there for convenience and in effect will be ignored by InputData initialization. Each subsequent row contains a labeled input pattern. Each row is organized into a sequence of entries, whereby the first entry contains the pattern label, and the following entries encode the actual input/target data. A sequence of input data comes first, followed by a sequence of target data. The example below is from the training data file for the example auto-encoder network (spaces are added for readability):

```
inp_label,  input,             target
p1,         1,0,0,0,0,0,0,0,   1,0,0,0,0,0,0,0
p2,         0,1,0,0,0,0,0,0,   0,1,0,0,0,0,0,0
p3,         0,0,1,0,0,0,0,0,   0,0,1,0,0,0,0,0
p4,         0,0,0,1,0,0,0,0,   0,0,0,1,0,0,0,0
p5,         0,0,0,0,1,0,0,0,   0,0,0,0,1,0,0,0
p6,         0,0,0,0,0,1,0,0,   0,0,0,0,0,1,0,0
p7,         0,0,0,0,0,0,1,0,   0,0,0,0,0,0,1,0
p8,         0,0,0,0,0,0,0,1,   0,0,0,0,0,0,0,1
```
**Importantly, each row must end with an implicit newline character "`\n`", not a comma "`,`"**.

## `InputData` class
It is straightforward to create an `InputData` instance from a well-organized data file. You need to specify a few parameters to make it work:
- **`num_epochs`** : the number of epochs you are planning to train/test the model for. An epoch is a single iteration inside which a network processes the entire data set (typically seeing each pattern exactly once).
- **`batch_size`** : the size of a subset (mini-batch) of the data set. Together with `data_len`, this parameter determines the number of weight updates per epoch of training. Specifically, within a single epoch of training, weights will be updated `data_len / batch_size` times. For example, if `batch_size == data_len`, the network will accumulate gradients for each training pattern and perform a weight update once per epoch. In contrast, if `batch_size == 1`, each training pattern will cause a weight update, making the order of training examples consequential. Moreover, if `(batch_size > 1 & batch_size < data_len)`, several training patterns will cause a weight update and there will be more than one update per epoch. Thus, `batch_size` must divide `data_len`, that is, the remainder of `data_len / batch_size` must be zero.
- **`data_len`** : the number of input patterns in the data set. Together with `batch_size`, this parameter determines the number of weight updates per epoch of training (see `batch_size`).
- **`inp_size`** : the number of input data points (same as the size of input layer). You can pass a list or a tuple of integers if you have multiple input placeholders. Note that the order of input sizes must correspond to the order of input placeholders when an `FFBP.Model` class is instantiated with multiple input placeholders (see tutorials on [connecting layers](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/building_models/build_model.ipynb) and [training a model](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/building_models/train_model.ipynb)). For example `inp_size = (2,6)` will devide the row of 8 input items into two sub-rows of sizes 2 and 6.
- **`targ_size`** : the number of target data points (same as the size of output layer).
-  **`shuffle_seed`** : (*default*=`None`) the seed for a random number generator that dictates the pseudorandom shuffling of input patterns. If `None`, input patterns will be fed in the same order (top to bottom, unshuffled) as they appear in the csv file. If negative (e.g. `shuffle_seed = -1`), the shuffling of examples will be irreproducible.

## Example: [Enqueueing](https://www.tensorflow.org/versions/r0.12/api_docs/python/io_ops/queues) data for training and testing
In the example below, we set up training and testing data for a training loop simulation. **Note** that for testing, `batch_size == 1` since we want to compute gradient information individually for each training example. Also note that we don't want to shuffle testing patterns (so ommit the `shuffle_seed` parameter or set it to `None`).

Feel free to change the parameters and observe the effects in the output.

In [ ]:
import tensorflow as tf
import FFBP
tf.logging.set_verbosity(tf.logging.ERROR) # Prevent unwanted logging messages by tensorflow

NUM_EPOCHS = 2

FFBP_GRAPH = tf.Graph()

with FFBP_GRAPH.as_default():
    
    # Create data for training
    TRAIN_DATA = FFBP.InputData(
        path_to_data_file = 'auto_data_train.txt',
        num_epochs = NUM_EPOCHS,
        batch_size = 4,
        data_len = 8,
        inp_size = 8,
        targ_size = 8,
        shuffle_seed = -1,
    )

    # Create data for testing
    TEST_DATA = FFBP.InputData(
        path_to_data_file = 'auto_data_test.txt',
        num_epochs = NUM_EPOCHS,
        batch_size = 1,
        inp_size = 8,
        targ_size = 8,
        data_len = 15,
    )

# Simulate training loop
with tf.Session(graph=FFBP_GRAPH) as sess:
        # Initialize variables
        sess.run(tf.local_variables_initializer())
        
        # create coordinator and start queue runners
        coordinator = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coordinator)

        for i in range(NUM_EPOCHS):
            print('\nEPOCH {}:\n'.format(i))
            
            # Mock-test model in a test mini-loop
            print('  TESTING:')
            for j in range(TEST_DATA.data_len):
                example = sess.run(TEST_DATA.examples_batch)
                print('    testing pattern {}: \'{}\' {}'.format(
                    j, example[0][0].decode('UTF-8'),
                    ', '.join([str(x) for x in example[1:-1]])))

            # Mock-train model in a train mini-loop
            print('  TRAINING:')
            num_updates = TRAIN_DATA.data_len // TRAIN_DATA.batch_size
            for k in range(num_updates):
                examples_batch = sess.run(TRAIN_DATA.examples_batch)
                print(
                    '    processing mini-batch {}/{}: {}'.format(
                        k+1, num_updates, 
                        [x.decode('UTF-8') for x in examples_batch[0]]
                    )
                )

                [print('\t' + str(x).replace('\n', '\n\t')) for x in examples_batch[1:-1]]

        coordinator.request_stop()
        coordinator.join(threads)